In [4]:
#################
## MYSQL STUFF ##
#################

In [1]:
import os
import mysql.connector
from mysql.connector import errorcode
from pathlib import Path
import json
import logging


currentWorkingDirectory = os.path.abspath('')
secretFile = json.load(open(currentWorkingDirectory+'/config/secrets.json'))
dbhost = secretFile['dbhost']
dbname = secretFile['db']
dbuser = secretFile['dbuser']
dbpass = secretFile['dbpass']


In [2]:
try:
    cnx = mysql.connector.connect(user=dbuser, password=dbpass,
                                  host=dbhost,
                                  database=dbname)
    print("connection successfully established.")
    
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Invalid user credentials.")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)
    cnx.close()

connection successfully established.


In [3]:
cnx.close()

In [4]:
######################
## REDDIT API STUFF ##
######################

In [5]:
import os
from pathlib import Path
import json
import logging
import praw

currentWorkingDirectory = os.path.abspath('')
secretFile = json.load(open(currentWorkingDirectory+'/config/secrets.json'))

use_script=secretFile['use_script']
client_secret=secretFile['client_secret']
user_agent=secretFile['user_agent']
username=secretFile['username']
password=secretFile['password']

In [6]:
reddit = praw.Reddit(client_id = use_script, 
                     client_secret = client_secret,
                     user_agent = user_agent,
                     username = username,
                     password = password)

In [7]:
subreddit_memes = reddit.subreddit("dankmemes")
hot_memes = subreddit_memes.hot(limit=5)

In [8]:
type(hot_memes)

praw.models.listing.generator.ListingGenerator

In [9]:
for i in hot_memes:
    print(i.title, i.url)

ASTRONOMIA COFFIN DANCE IS YOUR MARCH 2020 MEME OF THE MONTH https://i.imgur.com/X9SDUVN.gif
Did I miss anyone? https://i.redd.it/gkn1xu1cfcv41.gif
Watch where you're going https://i.redd.it/f0zkm19k9dv41.gif
The real battle royale https://i.redd.it/nvnb098z3cv41.jpg


In [1]:
######################
##### QUOTE API ######
######################

In [4]:
import requests
import json
url = ' https://opinionated-quotes-api.gigalixirapp.com/v1/quotes'
response = requests.get(url).json()

quote = response['quotes'][0]['quote']
author = response['quotes'][0]['author']
print(f"quote:{quote}\nauthor:{author}")

quote:The fact that one can lose one's sense of self in an ocean of tranquility does not mean that one's consciousness is immaterial or that it presided over the birth of the universe.
author:Sam Harris


In [5]:
######################
##### JOKE API #######
######################

In [64]:
jokeurl = 'https://sv443.net/jokeapi/v2/joke/Miscellaneous?blacklistFlags=racist,sexist'
payload = {}
headers = {'Accept':'application/json'}

response = requests.request("GET", jokeurl, headers=headers, data=payload).json()

In [65]:
response

{'category': 'Miscellaneous',
 'type': 'twopart',
 'flags': {'nsfw': False,
  'religious': False,
  'political': False,
  'racist': False,
  'sexist': False},
 'setup': 'Two guys walked into a bar.',
 'delivery': 'The third guy ducked.',
 'id': 170,
 'error': False}

In [66]:
print(response['setup'], response['delivery'])

Two guys walked into a bar. The third guy ducked.


In [47]:
#############################
##### TRUMP QUOTE API #######
#############################

trumpurl = 'https://api.tronalddump.io/random/quote'
response = requests.get(trumpurl).json()
response['value']

'.@lindseygraham, who had zero in his presidential run before dropping out in disgrace- saying the most horrible things about me on Fox News.'